In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-09-14 16:40:17.391841


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Collections

In [ ]:
%load_ext autoreload
%autoreload
col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (Parallel)

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    arts.parseArtistModValExtraFiles(modVal)


def parseArtistsParallel(nProcs=4, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel()
elapsed(start2, cmt2)
#arts.buildMetadata(force=False, doAlbums=False)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sat Sep 14, 2019 22:07:30 for Parsing All Artists
Current Time is Sat Sep 14, 2019 22:07:30 for Parallel Artist Parsing
Parsing Artist Files For ModVal 14
Parsing Artist Files For ModVal 7
Parsing Artist Files For ModVal 21
Parsing Artist Files For ModVal 0


# New Artists From Albums

In [4]:
toget = getFile('/Users/tgadfort/Documents/code/music/toget.p')
toget2 = getFile('/Users/tgadfort/Documents/code/music/toget2.p')
toget.update(toget2)

for ia,(artistID,artistRef) in enumerate(toget.items()):
    url = arts.getArtistURL(artistRef)
    savename = arts.getArtistSavename(artistID)
    try:
        arts.downloadArtistURL(url, savename, debug=True, force=True)
    except:
        continue
    if ia % 10 == 0:
        print("\nProcessed {0}/{1} Files\n".format(ia,len(toget)))

/Volumes/Music/Discog/artists/38/1115338.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1115338-Ren%C3%83%C2%A9-Simard?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/38/1115338.p
Saving /Volumes/Music/Discog/artists/38/1115338.p
  --> This file is 46.5kB.
Done. Sleeping for 2 seconds

Processed 0/5307 Files

/Volumes/Music/Discog/artists/38/1208438.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1208438-Ch%C3%83%C2%A9-Kropp?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/38/1208438.p
Saving /Volumes/Music/Discog/artists/38/1208438.p
  --> This file is 16.9kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/38/1538538.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1538538-S%C3%83%C2%A9rgio-Ricardo?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/38/1538538.p
Saving /Volumes/Music/Discog/artists

Saving /Volumes/Music/Discog/artists/38/791138.p
  --> This file is 20.9kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/38/917438.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/917438-Tolcsvay-B%C3%83%C2%A9la?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/38/917438.p
Saving /Volumes/Music/Discog/artists/38/917438.p
  --> This file is 19.5kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/0/1315400.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1315400-R%C3%83%C2%A9gis-Boulard?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/0/1315400.p
Saving /Volumes/Music/Discog/artists/0/1315400.p
  --> This file is 18.1kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/0/1474200.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1474200-Z%C3%83%C2%A9-Da-C%C3%83%C2%A2mara?sort=year%2Casc&limit=500
                   Sav

Saving /Volumes/Music/Discog/artists/0/5273900.p
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/0/5307300.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/5307300-Tarzan-Pep%C3%83%C2%A9?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/0/5307300.p
Saving /Volumes/Music/Discog/artists/0/5307300.p
  --> This file is 15.8kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/0/748100.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/748100-Z%C3%83%C2%A9-Carlos?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/0/748100.p
Saving /Volumes/Music/Discog/artists/0/748100.p
  --> This file is 40.2kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/0/752600.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/752600-Guy-L%C3%83%C2%A9onard?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/

Saving /Volumes/Music/Discog/artists/1/5587201.p
  --> This file is 15.6kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/1/613501.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/613501-Xanton%C3%83%C2%A9-Blacq?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/1/613501.p
Saving /Volumes/Music/Discog/artists/1/613501.p
  --> This file is 20.3kB.
Done. Sleeping for 2 seconds
Now Downloading in Artists(): https://www.discogs.com/artist/970901-An%C3%83%C2%A9?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/1/970901.p
  File ID != Artist ID. Renaming to /Volumes/Music/Discog/artists/25/4885825.p
Saving /Volumes/Music/Discog/artists/25/4885825.p
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/10/1037810.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1037810-Yves-Honor%C3%83%C2%A9?sort=year%2Casc&limit=500
                  

Saving /Volumes/Music/Discog/artists/10/830210.p
  --> This file is 17.4kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/10/9110.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/9110-J%C3%83%C2%A9r%C3%83%C2%B4me-Pacman?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/10/9110.p
Saving /Volumes/Music/Discog/artists/10/9110.p
  --> This file is 24.7kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/10/991710.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/991710-G%C3%83%C2%A9rard-Kr%C3%83%C2%A9mer?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/10/991710.p
Saving /Volumes/Music/Discog/artists/10/991710.p
  --> This file is 23.2kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/11/100711.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/100711-Gallifr%C3%83%C2%A9?sort=year%2Casc&limit=500
                   Saving

...


Processed 5200/5307 Files

/Volumes/Music/Discog/artists/81/252081.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/252081-Sj%C3%83%C2%B8lmord?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/81/252081.p
Saving /Volumes/Music/Discog/artists/81/252081.p
  --> This file is 16.4kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/81/396681.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/396681-Kai-Rob%C3%83%C2%B8le?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/81/396681.p
Saving /Volumes/Music/Discog/artists/81/396681.p
  --> This file is 63.8kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/81/4961181.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/4961181-Hans-Hendrik-Nordstr%C3%83%C2%B8m?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/81/4961181.p
Saving /Volumes/Music/Discog/artist

/Volumes/Music/Discog/artists/85/2589185.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/2589185-Amund-Sj%C3%83%C2%B8lie-Sveen?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/85/2589185.p
Saving /Volumes/Music/Discog/artists/85/2589185.p
  --> This file is 19.5kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/85/874285.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/874285-Gr%C3%83%C2%A6sr%C3%83%C2%B8dderne?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/85/874285.p
Saving /Volumes/Music/Discog/artists/85/874285.p
  --> This file is 18.4kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/86/1109886.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1109886-Espen-Mj%C3%83%C2%B8en?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/86/1109886.p
Saving /Volumes/Music/Discog/artists/86/1109886

Saving /Volumes/Music/Discog/artists/9/565309.p
  --> This file is 16.1kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/9/589209.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/589209-Fini-H%C3%83%C2%B8strup?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/9/589209.p
Saving /Volumes/Music/Discog/artists/9/589209.p
  --> This file is 17.7kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/90/1039490.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1039490-Bj%C3%83%C2%B8rn-Jensen-2?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/90/1039490.p
Saving /Volumes/Music/Discog/artists/90/1039490.p
  --> This file is 18.4kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/90/1154990.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/1154990-Tone-Bj%C3%83%C2%B8rnsen?sort=year%2Casc&limit=500
                   Saving as: /V

Saving /Volumes/Music/Discog/artists/93/2709393.p
  --> This file is 16.8kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/93/324493.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/324493-Bj%C3%83%C2%B8rn-Kjellemyr?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/93/324493.p
Saving /Volumes/Music/Discog/artists/93/324493.p
  --> This file is 19.2kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/93/3491593.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/3491593-Alexander-Lyngs%C3%83%C2%B8-Munk-Nielsen?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/93/3491593.p
Saving /Volumes/Music/Discog/artists/93/3491593.p
  --> This file is 16.5kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/93/464793.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/464793-C-V-J%C3%83%C2%B8rgensen?sort=year%2Casc&limit=500
           

/Volumes/Music/Discog/artists/97/2785197.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/2785197-Dick-Kays%C3%83%C2%B8?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/97/2785197.p
Saving /Volumes/Music/Discog/artists/97/2785197.p
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds

Processed 5290/5307 Files

/Volumes/Music/Discog/artists/97/3326897.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/3326897-Erik-T%C3%83%C2%B8raasen?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/97/3326897.p
Saving /Volumes/Music/Discog/artists/97/3326897.p
  --> This file is 15.6kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/97/4717397.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/4717397-Danmarks-Radios-B%C3%83%C2%B8rnekor?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/97/4717397.p
Saving /Volumes/Musi

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums

disc = discogs()
albs = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    print('\n{0}/{1}'.format(ia,len(refstoget)))
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
        sleep(2)
    #print(item)
#len(togetmap)

# Albums

In [ ]:
def parseAlbums(modVal, force=False):
    albs.parseAlbumModValFiles(modVal=modVal, force=force)
    
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)


def f_amp(modVals=100):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, range(100))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
albs.parseAlbumModValFiles(modVal='NAN', force=False)
f_amp()
elapsed(start2, cmt2)
albs.buildMetadata(force=False)
elapsed(start, cmt)

# New Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=3, mediaTypes=None, rand=True)

In [ ]:
for k,v in db.items():
    print(k,v)
    break

In [ ]:
f1="/Volumes/Music/Discog/albums/6/102506/482739.p"
#f2="/Volumes/Music/Discog/albums/6/102506/7303693.p"
#albData1 = alb.getData(f1)
f2="/Volumes/Music/Discog/albums/11/567511/3386791.p"
albData2 = alb.getData(f2)

In [ ]:
albData1.album

In [ ]:
for k,v in albData2.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
for k,v in albData1.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
from album import album
from fileUtils import getsize
alb = album()
for ifile in files:
    print(getsize(ifile))
    #alb.getData(ifile)

In [ ]:
for modVal in range(58,59):
    arts.parseArtistModValFiles(modVal=modVal, force=False)

dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
from time import sleep
refs = [dbCol.get('716158')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if True:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_mp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map(f, chunks)
 
    return sum(result)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
 
    while not result.ready():
        print("Running...")
        time.sleep(0.5)
 
    return sum(result.get())

In [ ]:
L = [{0: True}, {1: True}, {2: True}, {3: True}, {4: True}]

In [ ]:
{k: v for d in L for k, v in d.items()}

In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()
artistInfo = artist()
artistDBDir = disc.getArtistsDBDir()   

In [ ]:
len(files)

In [ ]:
from fileUtils import getBaseFilename
import time
from multiprocessing import Pool

def f(finput):
    a_list = finput["Data"]
    dbdata = finput["DB"]
    out = []
    for ifile in a_list:
        discID = getBaseFilename(ifile)
        if dbdata.get(discID) is None:
            out.append(ifile)
            
    return out

def fDB(modVal):
    saveIt = arts.parseArtistModValFiles(modVal, force=True)
    
    if False:
        from searchUtils import findExt
        files = findExt(setDir(artistDir, str(modVal)), ext="*.p")
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        dbdata = getFile(dbname, version=3)

        out = []
        saveIt = 0
        for ifile in files:
            discID = getBaseFilename(ifile)
            if dbdata.get(discID) is None:
                info   = artistInfo.getData(ifile)
                dbdata[discID] = info
                saveIt += 1

        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
        
    return saveIt

 
def f_ampFiles(a_list, dbdata):
    nProcs = 1
    chunks = [a_list[i::nProcs] for i in range(nProcs)]

    for c in chunks:
        print(len(c))
    
    pool = Pool(processes=nProcs)
    
    
 
    finputs = [{"Data": chunk, "DB": dbdata} for chunk in chunks]
    result = pool.map_async(f, finputs)
 
    start, cmt = clock("Processing {0} files using {1} processes".format(len(files), nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()



 
def f_amp(modVals):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(fDB, range(3,modVals))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
from timeUtils import clock, elapsed
retval = f_amp(100)
#flat_list = [item for sublist in retval for item in sublist]
#print(len(flat_list))

In [ ]:
dbname = setFile(artistDBDir, "{0}-DB.p".format(67))            
dbdata = getFile(dbname, version=3)

In [ ]:
modVals = 1
nProcs = 1
start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
print("Runing .",end="")
retval = fDB(modVals)
print("")
elapsed(start, cmt)


In [ ]:
retval

In [ ]:
vals="""1,708,466 US
1,708,466 UK
1,154,284 Germany
637,261 France
493,079 Japan
490,698 Italy
434,681 Canada
433,543 Europe
416,965 Netherlands
339,738 Spain
320,714 Unknown
259,587 Russia
259,579 Australia
189,431 Sweden
164,278 Belgium
149,548 Brazil
131,106 Greece
124,488 Poland
121,249 Jamaica
105,887 Finland
86,538 Switzerland
85,145 USSR
80,203 Mexico
79,972 Denmark
77,131 Portugal
75,979 Norway
75,533 UK & Europe
67,292 Austria
63,672 Argentina"""

In [ ]:
countries = [" ".join(x.split()[1:]) for x in vals.split("\n")]
saveFile(idata=countries, ifile="countries.p")

In [ ]:
countries